<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 9:** LangServe and Assessment</font>

<br>

## LangServe Server Setup

This notebook is a playground for those interested in developing interactive web applications using LangChain and [**LangServe**](https://python.langchain.com/docs/langserve). The aim is to provide a minimal-code example to illustrate the potential of LangChain in web application contexts.

This section provides a walkthrough for setting up a simple API server using LangChain's Runnable interfaces with FastAPI. The example demonstrates how to integrate a LangChain model, such as `ChatNVIDIA`, to create and distribute accessible API routes. Using this, you will be able to supply functionality to the frontend service's [**`frontend_server.py`**](frontend/frontend_server.py) session, which strongly expects:
- A simple endpoint named `:9012/basic_chat` for the basic chatbot, exemplified below.
- A pair of endpoints named `:9012/retriever` and `:9012/generator` for the RAG chatbot.
- All three for the **Evaluate** utility, which will be required for the final assessment. *More on that later!*

**IMPORTANT NOTES:**
- Make sure to click the square ( $\square$ ) button twice to shut down an active FastAPI cell. The first time might fall through or trigger a try-catch routine on an asynchronous process.
- If it still doesn't work, do a hard restart on this notebook by using **Kernel -> Restart Kernel**.
- When a FastAPI server is running in your cell, expect the process to block up this notebook. Other notebooks should not be impacted by this. 

<br>

### **Part 1:** Delivering the /basic_chat endpoint

Instructions are provided for launching a `/basic_chat` endpoint both as a standalone Python file. This will be used by the frontend to make basic decision with no internal reasoning.

In [5]:
%%writefile server_app.py
# https://python.langchain.com/docs/langserve#server
from fastapi import FastAPI
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langserve import add_routes

## May be useful later
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnablePassthrough
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_community.document_transformers import LongContextReorder
from functools import partial
from operator import itemgetter

from langchain_community.vectorstores import FAISS

## TODO: Make sure to pick your LLM and do your prompt engineering as necessary for the final assessment
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
instruct_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
#convstore = default_FAISS()
#retriever = new_db.as_retriever()
#docs = new_db.similarity_search("Testing the index")
#print(docs[0].page_content[:1000])

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

## PRE-ASSESSMENT: Run as-is and see the basic chain in action

add_routes(
    app,
    instruct_llm,
    path="/basic_chat",
)

## ASSESSMENT TODO: Implement these components as appropriate
#missing vectorstore, smth that we are using to format the input going to the model

add_routes(
    app,
    RunnableLambda(lambda x: chat_prompt.invoke(x) | instruct_llm | StrOutputParser()),
    path="/generator",
)

add_routes(
    app,
    RunnableLambda(lambda x: docstore.as_retriever(x)),
    path="/retriever",
)

## Might be encountered if this were for a standalone python file...
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=9012)

Overwriting server_app.py


In [ ]:
## Works, but will block the notebook.
!python server_app.py  

## Will technically work, but not recommended in a notebook. 
## You may be surprised at the interesting side effects...
# import os
# os.system("python server_app.py &")

INFO:     Started server process [799]
INFO:     Waiting for application startup.

     __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
    |  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
    |  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
    |  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
    |  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
    |_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|
    
LANGSERVE: Playground for chain "/basic_chat/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /basic_chat/playground/
LANGSERVE:
LANGSERVE: Playground for chain "/retriever/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /retriever/playground/
LANGSERVE:
LANGSERVE: Playground for chain "/generator/" is live

<br>

### **Part 2:** Using The Server:

While this cannot be easily utilized within Google Colab (or at least not without a lot of special tricks), the above script will keep a running server tied to the notebook process. While the server is running, do not attempt to use this notebook (except to shut down/restart the service).

In another file, however, you should be able to access the `basic_chat` endpoint using the following interface:

```python
from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser

llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()
for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')
```

**Please try it out in a different file and see if it works!**


<br>

### **Part 3: Final Assessment**

**This notebook will be used to completing the final assessment!** When you have otherwise finished the course, we recommend cloning this notebook, getting the frontend open in a new tab, and implement the Evaluate functionality by implementing the `/generator` and `/retriever` endpoints above! For a quick link to the frontend, run the cell below:

In [7]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Gradio Frontend ></h2></a>';

<IPython.core.display.Javascript object>

<hr>
<br>

#### **Assessment Hint:** 
Note that the following functionality is already implemented in the frontend microservice. 

```python
## Necessary Endpoints
chains_dict = {
    'basic' : RemoteRunnable("http://lab:9012/basic_chat/"),
    'retriever' : RemoteRunnable("http://lab:9012/retriever/"),  ## For the final assessment
    'generator' : RemoteRunnable("http://lab:9012/generator/"),  ## For the final assessment
}

basic_chain = chains_dict['basic']

## Retrieval-Augmented Generation Chain

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign(
        {'context' : itemgetter('input') 
        | chains_dict['retriever'] 
        | LongContextReorder().transform_documents
        | docs2str
    })
)

output_chain = RunnableAssign({"output" : chains_dict['generator'] }) | output_puller
rag_chain = retrieval_chain | output_chain
```

**To conform to this endpoint ingestion strategy, make sure not to duplicate pipeline functionality and only deploy the features that are missing!**

----

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>